# COMPSCI 682 – FINAL PROJECT
### Problem: Conventional City Classification

Below **Imports**

In [1]:
import os
import PIL
import cv2

import random
import numpy as np
import scipy.io
import matplotlib.pyplot as plt

# TORCH
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T
import torch.nn.functional as F

from sklearn.cluster import KMeans

# UTILS
from vis_utils import visualize_grid

Below **Preproccess Data** 


**Step 0.1**: Preprocesses and reorganize the dataset

The dataset images are denoted as follows (*from the Dataset website*):


> *The name of the images has the following format: XXXXXX_Y.jpg. XXXXXX is the identifier of the placemark. There are total number of 10343 placemarks in this dataset, so XXXXXX ranges from 000001 to 10343.
Y is the identifier of the view. 1, 2, 3 and 4 are the side views and 5 is the upward view. 0 is the view with markers overlaid (explained above). Thus, there are total number of 6 images per placemark.*




In [2]:
from utils.data_utils import Macro_Classification


pitts_folder = "part1"
ny_folder = "part5"

data_folders = [pitts_folder, ny_folder]
class_data = Macro_Classification()
class_data.load_data(folder_paths=data_folders)
class_data.label_data()
class_data.sample_dataset(4)


KeyboardInterrupt: 

**Step 1.** Preprocesses Data for Model

In [ ]:
# CONVERT numpy array to TENSOR
X = class_data.data[0]
y = class_data.labels

print("num 0s [Pittsburgh]: ", np.sum(y == 0))
print("num 1s [Orlando]: ", np.sum(y == 1))
print("num 2s [New York]: ", np.sum(y == 2))
print("data and labels equal: ", len(X) == len(y))

print(X.shape)

**Initialize** Data Loaders

In [ ]:
from utils.torch_utils import process_data

labels_to_exclude = [2]
loader_train, loader_val = process_data(X, y, exclude_labels=labels_to_exclude)

**Step 2.** Build Convolutional Model

In [ ]:
from models.cnn_classifier import CNN_Classifier

cnn_class = CNN_Classifier(num_classes=2, output_dims=2430)
cnn_class.set_up_train(loader_train, loader_val)

**Step 3**. Train Model

In [ ]:
optimizer = optim.SGD(cnn_class.model.parameters(), lr=0.0008,
                     momentum=0.9, nesterov=True)

evals = cnn_class.train_model(optimizer, epochs=10, lr=0.0008, print_every=10)

In [ ]:
'''
plt.plot(train_acc[:1000], label = "train")
plt.plot(valid_acc[:1000], label = "validation")
plt.legend()
plt.show()
'''
import json

with open("cnc_evals.json", "w") as outfile:
    json.dump(evals, outfile)

**Step 4.** Saliencey Maps 

In [ ]:
def compute_saliency_maps(X, y, model):
    """
    Compute a class saliency map using the model for images X and labels y.

    Input:
    - X: Input images; Tensor of shape (N, 3, H, W)
    - y: Labels for X; LongTensor of shape (N,)
    - model: A pretrained CNN that will be used to compute the saliency map.

    Returns:
    - saliency: A Tensor of shape (N, H, W) giving the saliency maps for the input
    images.
    """
    # Make sure the model is in "test" mode
    model.eval()
    
    # Make input tensor require gradient
    X.requires_grad_()
    
    saliency = None
    ##############################################################################
    # TODO: Implement this function. Perform a forward and backward pass through #
    # the model to compute the gradient of the correct class score with respect  #
    # to each input image. You first want to compute the loss over the correct   #
    # scores (we'll combine losses across a batch by summing), and then compute  #
    # the gradients with a backward pass.                                        #
    ##############################################################################
    # COMPUTE model scores 
    scores = model.forward(X)
    
    # COMPUTE the loss
    rel_scores = scores.gather(1, y.view(-1, 1)).squeeze()
    
    loss_func = nn.CrossEntropyLoss()
    loss = loss_func(scores, y)
    loss.backward()
    
    # GET X derivative
    X_derivs = X.grad
    print(X_derivs.shape)
    saliency, _ = torch.max(torch.abs(X_derivs), 1)
    print(saliency.shape)
        
    # MAP instances to CORRECT label scores
    ##############################################################################
    #                             END OF YOUR CODE                               #
    ##############################################################################
    return saliency

def show_saliency_maps(X, y):
    # Convert X and y from numpy arrays to Torch Tensors
    X_tensor = torch.Tensor(X)
    y_tensor = torch.LongTensor(y)

    # Compute saliency maps for images in X
    saliency = compute_saliency_maps(X_tensor, y_tensor, model)

    # Convert the saliency map from Torch Tensor to numpy array and show images
    # and saliency maps together.
    saliency = saliency.numpy()
    N = X.shape[0]
    for i in range(N):
        plt.subplot(2, N, i + 1)
        image = np.moveaxis(X[i], (0, 1, 2), (2, 1, 0))
        plt.imshow(image)
        plt.axis('off')
        plt.title(class_names[y[i]])
        plt.subplot(2, N, N + i + 1)
        plt.imshow(saliency[i], cmap=plt.cm.hot)
        plt.axis('off')
        plt.gcf().set_size_inches(12, 5)
    plt.show()


In [ ]:
import random

class_names = ["New York","Pittsburgh", "Orlando"]


def get_random_images(vis_data, num_images):
    choices = np.zeros(shape=(num_images, 3, 100, 100))
    choice_labels = np.zeros(shape=(num_images), dtype=int)
    for i in range(num_images):
        rand_index = random.randint(0, len(data))
        X, y = data[rand_index] 
        choices[i] = torch.Tensor(X)
        choice_labels[i] = y
        
    return choices, choice_labels
    
sample_X, sample_y = get_random_images(data, 4)
print(sample_X.shape)
      
plt.figure(figsize=(100, 100))
for i in range(len(sample_X)):
    plt.subplot(1, 5, i + 1)
    image = np.moveaxis(sample_X[i], (0, 1, 2), (2, 1, 0))
    plt.imshow(image)

    plt.title(class_names[sample_y[i]])
    plt.axis('off')
plt.gcf().tight_layout()

show_saliency_maps(sample_X, sample_y)